# Evaluating cellpose hyperparameters

NOTE : use provided env for this notebook.

Evaluates performance on the images generated by `generate_cellpose_hyperparameters.ipynb`

In [ ]:
from stardist import matching_dataset